In [5]:
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import sys


In [6]:
def compile_arduino_code(code: str, sketch_name="MySketch"):
    folder = sketch_name
    filename = f"{sketch_name}.ino"
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, filename)

    with open(file_path, "w") as f:
        f.write(code)

    try:
        result = subprocess.run(
            ["arduino-cli", "compile", "--fqbn", "arduino:avr:uno", folder],
            capture_output=True,
            text=True
        )
    except FileNotFoundError:
        return False, "arduino-cli not found or not in PATH"

    log = "\n".join(result.stderr.splitlines()[:20]).strip() 
    return result.returncode == 0, log

In [7]:
# --- Load model ---
from transformers import TextGenerationPipeline

model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)



code_fixer = TextGenerationPipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    pad_token_id=tokenizer.pad_token_id
)

Fetching 2 files:  50%|█████     | 1/2 [34:45<34:45, 2085.80s/it]


OSError: deepseek-ai/deepseek-coder-6.7b-instruct does not appear to have a file named model-00002-of-00002.safetensors. Checkout 'https://huggingface.co/deepseek-ai/deepseek-coder-6.7b-instruct/tree/main'for available files.

In [ ]:
def trim_context_snippets(snippets, tokenizer, max_context_tokens):
    combined = ""
    for snippet in snippets:
        temp = combined + "\n\n" + snippet
        tokens = tokenizer(temp, return_tensors="pt", truncation=False).input_ids
        if tokens.shape[1] > max_context_tokens:
            break
        combined = temp
    return combined

In [ ]:
from prompt_template import correct_code
import re

def extract_code_block(text):
    match = re.search(r"```(?:arduino)?\s*(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()  # fallback

def fix_code(code, error, context_snippets, model, tokenizer, max_total_tokens=2048, max_new_tokens=256):
    max_context_tokens = int(max_total_tokens * 0.75)
    context = trim_context_snippets(context_snippets, tokenizer, max_context_tokens)

    base_prompt = correct_code(context, code, error)  

    input_ids = tokenizer(base_prompt, return_tensors="pt", padding=True, truncation=True)
    if input_ids["input_ids"].shape[1] + max_new_tokens > max_total_tokens:
        allowed = max_total_tokens - max_new_tokens
        input_ids["input_ids"] = input_ids["input_ids"][:, -allowed:]
        input_ids["attention_mask"] = input_ids["attention_mask"][:, -allowed:]

    output_ids = model.generate(
        input_ids=input_ids["input_ids"],
        attention_mask=input_ids["attention_mask"],
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id
    )
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return extract_code_block(decoded)



In [ ]:
def load_arduino_examples_from_repo(root_dir="arduino-examples/examples"):
    ino_files = []
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".ino"):
                path = os.path.join(subdir, file)
                try:
                    with open(path, "r", encoding="utf-8", errors="ignore") as f:
                        ino_files.append(f.read()[:5000])
                except:
                    pass
    return ino_files


In [ ]:
import re

def extract_clean_code(output: str, fallback: str) -> str:
    # Find the first ```arduino or ``` block
    
    match = re.search(r"```(?:arduino)?\s*(.*?)```", output, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # Otherwise, try to grab from last 'Corrected Code:' onward
    if "Corrected Code:" in output:
        return output.split("Corrected Code:")[-1].strip()

    # Fallback if all else fails
    return fallback


In [8]:
#def iterative_fix(original_code: str) -> str | None:
    #code = original_code
    #MAX_ATTEMPTS = 3
    #docs, embedder, index = build_faiss_index_from_arduino_examples("arduino-examples/examples")
    #for attempt in range(1, MAX_ATTEMPTS + 1):
    #ok, log = compile_arduino_code(code)
    #if ok:
        #print(f"✅ Compiled on attempt {attempt}")
     #   return code

    #print(f"❌ Attempt {attempt} failed, retrying…\n{log}", file=sys.stderr)
    #ctx = retrieve_context(log, embedder, index, docs)
    #raw = fix_code(code, log, ctx, model, tokenizer)
    #code = extract_clean_code(raw, code)
    #print("❌ Exhausted retries — no working sketch.")
    #return raw


In [9]:
def iterative_fix(original_code: str) -> str:
    code = original_code

    # Try compiling
    ok, log = compile_arduino_code(code)
    if ok:
        print("✅ Compilation succeeded — no correction needed.")
        return code

    print("❌ Compilation failed. Attempting to fix…\n", log)

    # Retrieve related Arduino examples
    docs, embedder, index = build_faiss_index_from_arduino_examples("arduino-examples/examples")
    ctx = retrieve_context(log, embedder, index, docs)

    # Generate a corrected version
    raw = fix_code(code, log, ctx, model, tokenizer)

    # Optional: clean the result to remove explanations or markdown
    fixed_code = extract_clean_code(raw, fallback=code)

    return fixed_code


In [10]:
def build_faiss_index_from_arduino_examples(local_path="arduino-examples/examples"):
    docs = load_arduino_examples_from_repo(local_path)
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedder.encode(docs).astype("float32") 
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return docs, embedder, index

def retrieve_context(query, embedder, index, docs, top_k=2):
    q_vec = np.array(embedder.encode([query]))
    D, I = index.search(q_vec, top_k)
    return [docs[i] for i in I[0]]

In [11]:
from pathlib import Path

buggy_code_path = Path("buggy_code.ino")
buggy_code = buggy_code_path.read_text(encoding="utf-8")

fixed = iterative_fix(buggy_code)
if fixed:
    print("\n—— Final working sketch ——\n")
    print(fixed)
else:
    print("No valid fix found after retries.")


❌ Compilation failed. Attempting to fix…
 C:\Users\medaminekh\semester_project\MySketch\MySketch.ino: In function 'void setup()':
C:\Users\medaminekh\semester_project\MySketch\MySketch.ino:3:3: error: expected ';' before 'pinMode'
   pinMode(13, OUTPUT);  // missing comma between arguments
   ^~~~~~~
Error during build: exit status 1

—— Final working sketch ——

void setup() {
  Serial.begin(9600);
  pinMode(13, OUTPUT);  // added comma between arguments
}

void loop() {
  digitalWrite(13, HIGH);
  delay(1000);
}


### Explanation:
The error message indicates that the compiler is expecting a ';' after 'pinMode' but it's not. The missing comma after '13' in 'pinMode(13, OUTPUT);' is causing the error.

The corrected code adds a comma after '13' in 'pinMode(13, OUTPUT);' to fix the error. The 'HIGH' value is also added after 'digitalWrite(13, "HIGH");' to fix the error.
